## Imports and Loading Data

In [ ]:
import re

from langchain_upstage import ChatUpstage
from langchain_core.prompts import PromptTemplate
from langchain_upstage.embeddings import UpstageEmbeddings

In [ ]:
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
pip install wikipedia-api

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
import wikipediaapi

In [ ]:
UPSTAGE_API_KEY = api_key

## Defining functions and neccesary templates

Function to split the prompt into the question and options

In [11]:
def extract_question_options_answer(prompt):
    """
    Function that dynamically extracts questions, options, and answers
    """
    options_start = re.search(r"\([A-Z]\)\s", prompt)  # Find the location of the first option
    
    question = prompt[:options_start.start()].strip() if options_start else None
    
    options_match = re.findall(r"\([A-Z]\)\s.*?(?=\n|$)", prompt, re.DOTALL)
    options = options_match if options_match else []
    
    answer_match = re.search(r"\[ANSWER\]:\s*\((.)\)", prompt)
    correct_answer = answer_match.group(1).strip() if answer_match else None
    
    return question, options, correct_answer

Using LLM to generate category of the question

In [ ]:
# Define a prompt template
llm_category = ChatUpstage(api_key = UPSTAGE_API_KEY)
category_prompt_template = PromptTemplate.from_template(
    """
    You are given a question and its answer options. 
    Your task is to identify the category to which the question belongs to, out of the following categories:
    - Law  
    - Psychology  
    - Business  
    - Philosophy  
    - History  
    
    Instructions:
    1. Read the question and options carefully.
    2. Identify the category that best fits the question.
    3. Return only the category name (Law/Psycology/Business/Philosophy/History).
    ---
    Question: 
    {question}

    Options: 
    {options}
    """
)
category_chain = category_prompt_template | llm_category

Function to generate questions embeddings

In [13]:
# Initialize the Upstage embedding model
embedding_model = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large-query"  # Specify the model
)

In [20]:
def generate_question_embeddings(question, embedding_model):
    """
    Generate embeddings for a question and its words.
    """
    question_embedding = embedding_model.embed_documents([question])[0]

    words = question.split()
    word_embeddings = embedding_model.embed_documents(words)

    return question_embedding, words, word_embeddings

In [21]:
def extract_top_keywords(question_embedding, words, word_embeddings, top_n=3):
    """
    Extract top N keywords from the question based on cosine similarity with the question embedding.
    """
    similarities = cosine_similarity([question_embedding], word_embeddings)[0]
    top_indices = similarities.argsort()[-top_n:][::-1]
    top_keywords = [words[i] for i in top_indices]
    return top_keywords

Function to search keywords in wikipedia

In [16]:
wiki_wiki = wikipediaapi.Wikipedia(language='en', user_agent="MyApp/1.0 (threwja@gamil.com)")

In [18]:
def search_wikipedia(keyword):
    """
    Search Wikipedia for a given keyword and return the first paragraph of the summary.
    Args:
        keyword (str): The keyword to search on Wikipedia.
    Returns:
        str: The first paragraph of the Wikipedia page summary or an error message.
    """
    page = wiki_wiki.page(keyword)
    if page.exists():
        # Return the first paragraph of the summary
        return page.summary.split('\n')[0]
    else:
        return f"No information found for {keyword}"

Function to generate context from wikipedia

In [26]:
lemmatizer = WordNetLemmatizer()

In [19]:
def generate_context_from_wikipedia(keywords):
    """
    Generate a context by fetching information from Wikipedia for the given keywords.
    """
    context = []
    for keyword in keywords:
        summary = search_wikipedia(keyword)
        context.append(f"Keyword: {keyword}\n{summary}")
    return "\n\n".join(context)

In [27]:
def generate_context_from_wikipedia(keywords):
    """
    Fetch Wikipedia context for the given keywords.
    If a keyword has no result, try its lemmatized form.
    Args:
        keywords (list): List of keywords to search on Wikipedia.
    Returns:
        str: Context string containing summaries for the keywords.
    """
    context = ""

    for keyword in keywords:
        # Lemmatize the keyword
        lemmatized_keyword = lemmatizer.lemmatize(keyword)

        # Search for the lemmatized keyword first
        result = search_wikipedia(lemmatized_keyword)
        
        # If no information is found, fall back to the original keyword
        if "No information found" in result:
            result = search_wikipedia(keyword)

        # Add the result to the context
        context += f"Keyword: {keyword}\n{result}\n\n"

    return context

Defining the prompt template

In [ ]:
mmlu_llm = ChatUpstage(api_key = UPSTAGE_API_KEY)
mmlu_prompt_template = PromptTemplate.from_template(
    """
    You are a highly knowledgeable expert in {category}, renowned for precision in decision-making and logical reasoning.
    Your task is to analyze the following multiple-choice question and select the single best option based on your expertise.

    ### Instructions:
    1. Carefully read and understand the provided question and options.
    2. Use logical reasoning and domain-specific knowledge to identify the most accurate answer.
    3. Provide your answer in the following format:
       [ANSWER]: (option letter)

    ### Guidelines for Your Response:
    - Ensure your response is concise and strictly follows the requested format.
    - If the question is ambiguous or incomplete, explain why before providing an answer.
    - Only choose one option that best fits the context of the question.

    ### Format for Your Response:
    1. **Keywords Identified**: List the key concepts or keywords from the question.
    2. **Reasoning**: Briefly explain how the keywords help in selecting the best answer.
    3. **Final Answer**: Provide the single best option in the following format: [ANSWER]: (option letter)

    ---
    **Question:** 
    {question}

    **Options:** 
    {options}

    ---
    """
)
mmlu_chain = mmlu_prompt_template | mmlu_llm

In [21]:
# Generate category of the question
def generate_category_for_question(question, options):
    category_response = category_chain.invoke({"question": question, "options": options}).content
    return category_response

- Prompts the model for a response 5 times and chooses the most common returned response. 

- If there is no response returned, it will reprompt the model for another 5 responses, until the model returns a response.

- If maximum number of retries is exceeded, then use logical reasoning

In [22]:
# returns most common answer if consistency is above threshold
def get_consistent_answer(responses, consistency_threshold=0.8):
    
    answer_counts = {}

    # Count the number of times each answer appears in the list of responses
    for response in responses:
        match = re.search(r"\[ANSWER\]:\s*\(([A-Z])\)", response)
        if match:
            answer = match.group(1)
            answer_counts[answer] = answer_counts.get(answer, 0) + 1
    
    # Calculate the total number of responses and the most common answer
    total_responses = len(responses)
    most_common_answer = max(answer_counts, key=answer_counts.get, default=None)
    most_common_count = answer_counts.get(most_common_answer, 0)

    # Check if the most common answer is consistent enough (above the threshold)
    if most_common_count / total_responses >= consistency_threshold:
        return f"[ANSWER]: ({most_common_answer})"
    else:
        return None

In [1]:
def get_answer(context, question, options, category, max_retries=3):
    consistent_answer = None
    retries = 0

    while consistent_answer is None and retries < max_retries:
        individual_responses = []

        # Generate 5 independent responses
        for _ in range(5):  
            response = mmlu_chain.invoke({"context": context, "question": question, "options": options, "category": category, "temperature": 0}).content
            individual_responses.append(response)

        # Get the most consistent answer
        consistent_answer = get_consistent_answer(individual_responses)

        # Printing the generated responses and the consistent answer
        print(f"Generated Responses: {individual_responses}")
        print(f"Consistent Answer: {consistent_answer}")
        
        retries += 1

    if consistent_answer is None:
        print("No consistent answer found. Using the most recent logical response.")
        logical_response = individual_responses[-1]
        match = re.search(r"\[ANSWER\]:\s*\(([A-Z])\)", logical_response)
        if match:
            consistent_answer = f"[ANSWER]: ({match.group(1)})"
        else:
            print("No valid answer found in the logical reasoning response.")
            consistent_answer = None

    return consistent_answer

## Generating Response

In [19]:
def handle_mmlu(sampled_prompts):
    responses = []

    for prompt in sampled_prompts:
        # Split the prompt into question and options
        question, options_list, _ = extract_question_options_answer(prompt)
        options = "\n".join(options_list)
        print("Question:", question)
        print("Options:", options)

        # Generate the category for the question
        category = generate_category_for_question(question, options)
        print("Category:", category)

        # Step 1: Generate question embedding and word embeddings
        question_embedding, words, word_embeddings = generate_question_embeddings(
            question, embedding_model
        )

        # Step 2: Extract top 3 keywords
        top_keywords = extract_top_keywords(question_embedding, words, word_embeddings)
        print("Top Keywords:", top_keywords)

        # Step 3: Fetch Wikipedia context for the keywords
        wikipedia_context = generate_context_from_wikipedia(top_keywords)
        print("Wikipedia Context:\n", wikipedia_context)

        # Step 4: Use context in the LLM call
        context = f"### Context:\n{wikipedia_context}"
        print("Final Context:\n", context)

        # Step 5: Call get_answer with separate context
        consistent_answer = get_answer(context, question, options, category)

        print("Response:", consistent_answer)
        print("-------------------------------------------------------------\n")

        responses.append(consistent_answer)

    return responses